# 检索机器人

## Step1 读取faq数据

In [1]:
import pandas as pd

data = pd.read_csv("./data/law_faq.csv")
data.head()

,title,reply
0,在法律中定金与订金的区别订金和定金哪个受,“定金”是指当事人约定由一方向对方给付的，作为债权担保的一定数额的货币，它属于一种法律上的担...
1,盗窃罪的犯罪客体是什么，盗窃罪的犯罪主体,盗窃罪的客体要件本罪侵犯的客体是公私财物的所有权。侵犯的对象，是国家、集体或个人的财物，一般...
2,非法微整形机构构成非法经营罪吗,符合要件就有可能。非法经营罪，是指未经许可经营专营、专卖物品或其他限制买卖的物品，买卖进出口...
3,入室持刀行凶伤人能不能判刑,对于入室持刀伤人涉嫌故意伤害刑事犯罪，一经定罪，故意伤害他人身体的，处三年以下有期徒刑、拘役...
4,对交通事故责任认定书不服怎么办，交通事故损,事故认定书下发后，如果你对认定不满意，可在接到认定书3日内到上一级公安机关复议。


## Step2 加载模型

In [2]:
from dual_model_bert import DualModel

# 需要完成前置模型训练
dual_model = DualModel.from_pretrained("./dual_model/checkpoint-2000/")
dual_model = dual_model.cuda()
dual_model.eval()
print("匹配模型加载成功！")

匹配模型加载成功！


In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("D:/pretrained_model/models--hfl--chinese-macbert-base")

## Step3 将问题编码为向量

In [4]:
import torch

questions = data['title'].to_list()
vectors = []

with torch.inference_mode():
    for i in range(0, len(questions), 4):
        batch_sens = questions[i : i + 4]
        inputs = tokenizer(batch_sens, return_tensors='pt', max_length=128, truncation=True, padding=True)
        inputs = {k:v.to(dual_model.device) for k,v in inputs.items()}
        vector = dual_model.bert(**inputs)[1] # 拿到的是CLS token向量 [0]是句子的Bert表征
        vectors.append(vector)

vectors = torch.concat(vectors, dim=0).cpu().numpy() # Faiss 的 API 设计时假设输入数据是 NumPy 数组，因此为了保持一致性和避免潜在的问题，通常需要将 PyTorch 张量转换为 NumPy 数组。


In [7]:
vectors.shape

(18213, 768)

## Step4 创建索引

In [8]:
import faiss

index = faiss.IndexFlatIP(768)
faiss.normalize_L2(vectors)
index.add(vectors)
index


<faiss.swigfaiss.IndexFlatIP; proxy of <Swig Object of type 'faiss::IndexFlatIP *' at 0x0000013819681AA0> >

## Step5 对问题进行向量编码

In [14]:

def questions_vector(data):
    inputs = tokenizer(data, return_tensors='pt', padding=True, max_length=128, truncation=True)
    inputs = {k: v.to(dual_model.device)  for k,v in inputs.items()}
    vector = dual_model.bert(**inputs)[1]
    q_vector = vector.cpu().detach().numpy()
    return q_vector

question = "寻衅滋事"
__question_vector = questions_vector(question)

## Step6 向量匹配(召回)

In [16]:
data.values

array([['在法律中定金与订金的区别订金和定金哪个受',
        '“定金”是指当事人约定由一方向对方给付的，作为债权担保的一定数额的货币，它属于一种法律上的担保方式，目的在于促使债务人履行债务，保障债权人的债权得以实现。签合同时，对定金必需以书面形式进行约定，同时还应约定定金的数额和交付期限。给付定金一方如果不履行债务，无权要求另一方返还定金；接受定金的一方如果不履行债务，需向另一方双倍返还债务。债务人履行债务后，依照约定，定金应抵作价款或者收回。而“订金”目前我国法律没有明确规定，它不具备定金所具有的担保性质，可视为“预付款”，当合同不能履行时，除不可抗力外，应根据双方当事人的过错承担违约责任。'],
       ['盗窃罪的犯罪客体是什么，盗窃罪的犯罪主体',
        '盗窃罪的客体要件本罪侵犯的客体是公私财物的所有权。侵犯的对象，是国家、集体或个人的财物，一般是指动产而言，但不动产上之附着物，可与不动产分离的，例如，田地上的农作物，山上的树木、建筑物上之门窗等，也可以成为本罪的对象。另外，能源如电力、煤气也可成为本罪的对象。盗窃罪侵犯的客体是公私财物的所有权。所有权包括占有、使用、收益、处分等权能。这里的所有权一般指合法的所有权，但有时也有例外情况。根据《最高人民法院关于审理盗窃案件具体应用法律若干问题的解释》(以下简称《解释》)的规定：“盗窃违禁品，按盗窃罪处理的，不计数额，根据情节轻重量刑。盗窃违禁品或犯罪分子不法占有的财物也构成盗窃罪。”'],
       ['非法微整形机构构成非法经营罪吗',
        '符合要件就有可能。非法经营罪，是指未经许可经营专营、专卖物品或其他限制买卖的物品，买卖进出口许可证、进出口原产地证明以及其他法律、行政法规规定的经营许可证或者批准文件，以及从事其他非法经营活动，扰乱市场秩序，情节严重的行为。'],
       ...,
       ['老赖欠钱不还，可以用坐牢抵债吗',
        '肯定是要还的，判刑坐牢是针对他不还钱行为的惩罚，不是说坐牢抵债的，但如果一个人真的没钱你也没办法，只能看好对方的经济状况，一旦有钱的就得立马采取措施'],
       ['我男朋友把我姐夫的车给偷了，我姐夫报案了，一开始我不知道他偷车了我知道后帮他逃跑我有罪吗',
        '朋友。你男

In [ ]:
faiss.normalize_L2(__question_vector)
scores, indexes = index.search(__question_vector, 10)


In [19]:
indexes[0]

array([ 9828,  7489,  4416,  9819,  1672,  2273,   539,   230,  8508,
       10007], dtype=int64)

In [21]:
topk_result = data.values[indexes[0].tolist()]

In [27]:
topk_result.shape
topk_result[:,0]

array(['交通事故逃逸', '恶意诋毁他人', '寻衅滋事没有伤检得到对方的谅解', '交通事故肇事逃逸', '黑心高利贷优亿金融',
       '救助讨薪', '涉嫌寻衅滋事', '轻伤一级不赔钱判几年', '飞达暴力催收', '摩托车酒驾法律'], dtype=object)

## Step7 加载交互模型

In [ ]:
from transformers import BertForSequenceClassification

# 需要完成前置模型训练
corss_model = BertForSequenceClassification.from_pretrained("./cross_model/checkpoint-500/")
corss_model = corss_model.cuda()
corss_model.eval()
print("模型加载成功！")

## Step8 最终预测(排序)

In [ ]:
canidate = topk_result[:, 0].tolist()
ques = [question] * len(canidate)
inputs = tokenizer(ques, canidate, return_tensors="pt", padding=True, max_length=128, truncation=True)
inputs = {k: v.to(corss_model.device) for k, v in inputs.items()}
with torch.inference_mode():
    logits = corss_model(**inputs).logits.squeeze()
    result = torch.argmax(logits, dim=-1)
result

['交通事故逃逸',
 '恶意诋毁他人',
 '寻衅滋事没有伤检得到对方的谅解',
 '交通事故肇事逃逸',
 '黑心高利贷优亿金融',
 '救助讨薪',
 '涉嫌寻衅滋事',
 '轻伤一级不赔钱判几年',
 '飞达暴力催收',
 '摩托车酒驾法律']

In [ ]:
canidate_answer = topk_result[:, 1].tolist()
match_quesiton = canidate[result.item()]
final_answer = canidate_answer[result.item()]
match_quesiton, final_answer